### Step 1 - Pip install twilio
### Step 2 - Register for twilio account

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import os.path

In [2]:
#Connecing to Twilio
from twilio.rest import Client
account_sid = "XXXXX"
auth_token  = "XXXXX"
client = Client(account_sid, auth_token)

In [3]:
#Scrape Science mag
response = requests.get("https://www.scientificamerican.com/")
doc = BeautifulSoup(response.text, 'html.parser')

In [4]:
#Let's try to pull out the feature article
feature_article = doc.find('div', {'class': 'listing-feature__inner'})
feature_article

<div class="listing-feature__inner"><h1 class="t_feature-title"><a href="https://www.scientificamerican.com/article/why-people-dislike-really-smart-leaders/">Why People Dislike Really Smart Leaders</a></h1><p class="listing-feature__summary t_body">Those with an IQ above 120 are perceived as less effective, regardless of actual performance</p><p class="listing-feature__meta t_meta">8 hours ago —  Matthew Hutson</p></div>

In [5]:
#Let's create a dictionary for our feature story

feature_story = {
    'headline': feature_article.find("h1").text,
    'href': feature_article.find("a")["href"],
    'summary': feature_article.find("p", {"class": "listing-feature__summary t_body"}).text,
    'date': time.strftime("%Y-%m-%d-%H-%M")
}

feature_story

{'date': '2018-01-18-15-24',
 'headline': 'Why People Dislike Really Smart Leaders',
 'href': 'https://www.scientificamerican.com/article/why-people-dislike-really-smart-leaders/',
 'summary': 'Those with an IQ above 120 are perceived as less effective, regardless of actual performance'}

In [6]:
def send_sms():
    sms_msg = "NEW Feature from Scientific American: " + feature_story['headline']

    client.messages.create(
        to="XXXXX",
        from_="XXXXX",
        body=sms_msg,
        media_url=feature_story['href'])


In [7]:
#Check if the current path has a file called features history
#if it doesn't create one using our feature story

if os.path.isfile('features_history.csv'):
    features_history = pd.read_csv('features_history.csv')
    
else:
    df_feature_story = pd.DataFrame([feature_story])
    df_feature_story.to_csv('features_history.csv', index=False)

#Check to see if the last feature article on the list is different from the current one
#If it is different, append the new feature story to our history file and save it
if features_history['headline'].tail(1).item() != feature_story['headline']:
    features_history = features_history.append([feature_story])
    features_history.to_csv('features_history.csv', index=False)
    send_sms()
        
    

In [8]:
features_history

,date,headline,href,summary
0,2018-01-18-14-37,Why People Dislike Really Smart Leaders,https://www.scientificamerican.com/article/why...,Those with an IQ above 120 are perceived as le...
